In [5]:
import numpy as np
import pandas as pd
import itertools
import csv

### Conjuntos Frecuentes por Fuerza Bruta
Los conjuntos frecuentes o reglas de asociacion, son utilizadas en diversos lugares y areas de la mineria de datos, entre ellas la mineria de navegacion web, mineria de busqueda, mineria de canastas, etc.

Para esto, se utilizan los siguientes elementos:

* Un conjunto de Articulos I
* Un conjunto de Transacciones T
* Una regla de asociacion X -> Y
    * Donde X es un conjunto de antecedentes y Y es un conjunto de consecuentes, todos estos incluidos en el conjunto de articulos I
    


### Para esto utilizaremos ciertas metricas.
Soporte de una regla de asociacion

    support = ( X U Y ).count/n
Donde estamos marcando la cantidad de ocaciones que esta mostrandose esta regla de asociacion en las transacciones

Confianza de una regla

    confidence = ( X U Y ).count/X.count

Donde estamos evaluando la cantidad de veces que aparece esta regla de asociacion dependiendo de la cantidad de apariciones del antecedente ( X )
    
    

### Para este ejemplo utilizare el conjunto de objetos i, en este estara incluido:

* Salsa
* Tomate
* Verdura
* Tocino
* Coca Cola
* Sprite
* Mirinda
* Sabritas


In [6]:
i = ['salsa','tomate','verdura','tocino','coca cola','sprite','mirinda','sabritas']


### Y un conjunto de transacciones t con 7 transacciones descritas en la siguiente celda


In [7]:
t = []

t1 = set(['salsa','tomate','sabritas'])
t2 = set(['salsa','sabritas','coca cola','verdura'])
t3 = set(['salsa','sabritas','sprite','tocino'])
t4 = set(['salsa','verdura','tocino','mirinda'])
t5 = set(['verdura','tocino','sabritas','coca cola','mirinda'])
t6 = set(['salsa','tocino','sabritas'])
t7 = set(['mirinda','tocino','verdura'])
t = [t1,t2,t3,t4,t5,t6,t7]
t


[{'sabritas', 'salsa', 'tomate'},
 {'coca cola', 'sabritas', 'salsa', 'verdura'},
 {'sabritas', 'salsa', 'sprite', 'tocino'},
 {'mirinda', 'salsa', 'tocino', 'verdura'},
 {'coca cola', 'mirinda', 'sabritas', 'tocino', 'verdura'},
 {'sabritas', 'salsa', 'tocino'},
 {'mirinda', 'tocino', 'verdura'}]

### Obtenemos todos los subconjuntos X ( antecendentes ) , Y (consecuentes)

Al hacerlo por fuerza bruta podemos obtener todas las combinaciones de antecedentes ( X ) y consecuentes ( Y ) 

In [18]:
Y = {}
for l in range(1,len(i)):
    for subset in itertools.combinations(i, l):
        Y[subset] = list(set(i) - set(subset))

### Para cada antecedente y cada consecuente de este ( ahora llamado regla ), obtenemos todas las ocurrencias en las transacciones

In [17]:
rules = {}
for X in Y.keys():
    for item in Y[X]:
        for transaction in t:
            if set(list(X)+[item]).issubset(set(transaction)):
                rules[X,item] = rules.get((X,item),0) +1

### Obtenemos el soporte de cada regla

In [10]:
soporte = {}
for rule in rules.keys():
    soporte[rule] = rules[rule]/len(t)

### Mostramos unicamente las reglas de asociacion con un soporte del 40% o mayor

In [15]:
for rule in soporte.keys():
    if soporte[rule] >= .4:
        print(rule,soporte[rule])

(('salsa',), 'sabritas') 0.5714285714285714
(('salsa',), 'tocino') 0.42857142857142855
(('verdura',), 'tocino') 0.42857142857142855
(('verdura',), 'mirinda') 0.42857142857142855
(('tocino',), 'sabritas') 0.42857142857142855
(('tocino',), 'salsa') 0.42857142857142855
(('tocino',), 'verdura') 0.42857142857142855
(('tocino',), 'mirinda') 0.42857142857142855
(('mirinda',), 'tocino') 0.42857142857142855
(('mirinda',), 'verdura') 0.42857142857142855
(('sabritas',), 'tocino') 0.42857142857142855
(('sabritas',), 'salsa') 0.5714285714285714
(('verdura', 'tocino'), 'mirinda') 0.42857142857142855
(('verdura', 'mirinda'), 'tocino') 0.42857142857142855
(('tocino', 'mirinda'), 'verdura') 0.42857142857142855


Como podemos ver en el ejemplo anterior, la regla de asociacion con mayor soporte es:

X( Sabritas ) -> Y ( Salsa ) = soporte ( 0.5714 ) 

Al igual que la regla inversa, al ser fuerza bruta es posible encontrar relgas invertidas 

### Funcion que realiza lo anterior
Para facilitar la ejecucion con archivos csv o similares, se creo la siguiente funcion, esta regresa las relgas y soporte de todas las reglas, al igual que muestra las relgas que esten dentro del umbral


In [12]:
def ConjuntosFrecuentesFB(i,t,umbral):
    Y = {}
    rules = {}
    soporte = {}
    
    #Todos los antecedentes y sus posibles consecuentes
    for l in range(1,len(i)):
        for subset in itertools.combinations(i, l):
            Y[subset] = list(set(i) - set(subset))
    
    #Creacion de reglas y obtencion de sus ocurrencias
    for X in Y.keys():
        for item in Y[X]:
            for transaction in t:
                if set(list(X)+[item]).issubset(set(transaction)):
                    rules[X,item] = rules.get((X,item),0) +1
    #Obtencion del soporte
    for i in rules.keys():
        soporte[i] = rules[i]/len(t)
        if soporte[i] >= umbral:
            print(i,soporte[i])
    return rules,soporte
rules,soporte = ConjuntosFrecuentesFB(i,t,.5)

(('salsa',), 'sabritas') 0.5714285714285714
(('sabritas',), 'salsa') 0.5714285714285714


### Lectura de archivo transactions.csv
En este archivo se obtienen la lista de objetos ( i ) y la lista de transacciones ( t ) del archivo transactions.csv

In [13]:
def read_csv(file):
    i = []
    t = {}
    transactions = []
    with open('transactions.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            if row['Customer'] not in t.keys():
                t[row['Customer']] = []
            t[row['Customer']].append(row['Product'])
            i.append(row['Product'])
    i = list(set(i))
    for key in t.keys():
        transactions.append(set(t[key]))
    return i,transactions

### Items obtenidos

In [19]:
i,t = read_csv('transactions.csv')
i

['Steak',
 'Beer',
 'WhippedCream',
 'PeanutButter',
 'Yogurt',
 'Cheese',
 'Milk',
 'Fruit',
 'Jelly',
 'Vegetables',
 'ChocolateSauce',
 'Bread',
 'PotatoChips',
 'Soda']

### Transacciones obtenidas

In [20]:
t

[{'Bread', 'Fruit', 'Jelly', 'Milk', 'PeanutButter'},
 {'Bread',
  'Fruit',
  'Jelly',
  'Milk',
  'PeanutButter',
  'PotatoChips',
  'Soda',
  'Vegetables'},
 {'Beer', 'ChocolateSauce', 'Fruit', 'WhippedCream'},
 {'Bread', 'Fruit', 'Jelly', 'PotatoChips', 'Soda', 'Steak'},
 {'Fruit', 'Jelly', 'Milk', 'PeanutButter', 'Soda'},
 {'Bread', 'Fruit', 'Jelly', 'Milk', 'PotatoChips', 'Soda'},
 {'Fruit', 'Milk', 'PotatoChips', 'Soda'},
 {'Fruit', 'Milk', 'PeanutButter', 'Soda'},
 {'Cheese', 'Fruit', 'Yogurt'},
 {'Beer', 'Vegetables', 'Yogurt'}]

### Ejecucion con un umbral del 60%

In [21]:
rules, support = ConjuntosFrecuentesFB(i,t,.6)

(('Milk',), 'Fruit') 0.6
(('Fruit',), 'Milk') 0.6
(('Fruit',), 'Soda') 0.6
(('Soda',), 'Fruit') 0.6


### Ejecucion con un umbral del 40%


In [22]:
rules, support = ConjuntosFrecuentesFB(i,t,.4)

(('PeanutButter',), 'Milk') 0.4
(('PeanutButter',), 'Fruit') 0.4
(('Milk',), 'PeanutButter') 0.4
(('Milk',), 'Fruit') 0.6
(('Milk',), 'Jelly') 0.4
(('Milk',), 'Soda') 0.5
(('Fruit',), 'PeanutButter') 0.4
(('Fruit',), 'Milk') 0.6
(('Fruit',), 'Jelly') 0.5
(('Fruit',), 'Bread') 0.4
(('Fruit',), 'PotatoChips') 0.4
(('Fruit',), 'Soda') 0.6
(('Jelly',), 'Milk') 0.4
(('Jelly',), 'Fruit') 0.5
(('Jelly',), 'Bread') 0.4
(('Jelly',), 'Soda') 0.4
(('Bread',), 'Fruit') 0.4
(('Bread',), 'Jelly') 0.4
(('PotatoChips',), 'Fruit') 0.4
(('PotatoChips',), 'Soda') 0.4
(('Soda',), 'Milk') 0.5
(('Soda',), 'Fruit') 0.6
(('Soda',), 'Jelly') 0.4
(('Soda',), 'PotatoChips') 0.4
(('PeanutButter', 'Milk'), 'Fruit') 0.4
(('PeanutButter', 'Fruit'), 'Milk') 0.4
(('Milk', 'Fruit'), 'PeanutButter') 0.4
(('Milk', 'Fruit'), 'Jelly') 0.4
(('Milk', 'Fruit'), 'Soda') 0.5
(('Milk', 'Jelly'), 'Fruit') 0.4
(('Milk', 'Soda'), 'Fruit') 0.5
(('Fruit', 'Jelly'), 'Milk') 0.4
(('Fruit', 'Jelly'), 'Bread') 0.4
(('Fruit', 'Jelly'), 'S

### Conclusiones
Como podemos observar, este metodo es efectivo pero muy tardado ya que genera todas las reglas habidas y por haber incluso si estas no son incluidas en las transacciones

#### 160300153 - Vazquez Pompa Noe